Parse TLE to KeplerInputs

In [1]:
import sys
sys.path.append('/workspaces/ma-max-wendler/scripts/keplertraces')  # Add the directory to the search path

In [2]:
import tleparse

In [3]:
# inputs = tleparse($tlespath$), i.e.
path = "/workspaces/ma-max-wendler/scripts/keplertraces/tles/examples_requested_at_2023-10-23-11-06-08/iridium-NEXT_2023-10-22-21-39-19.txt"
tles = tleparse.read(path)

In [4]:
# objects of the TLE-tool that among other things parses TLE values
tle = tles[0]
tle

TLE(name='IRIDIUM 106', norad='41917', classification='U', int_desig='17003A', epoch_year=2023, epoch_day=295.91774854, dn_o2=1.91e-06, ddn_o6=0.0, bstar=6.0989e-05, set_num=999, inc=86.3949, raan=133.2138, ecc=0.0001982, argp=93.4819, M=266.6603, n=14.34216513, rev_num=35442)

For comparison: IRIDIUM 106 TLE

```
IRIDIUM 106
1 41917U 17003A   23295.91774854  .00000191  00000+0  60989-4 0  9995
2 41917  86.3949 133.2138 0001982  93.4819 266.6603 14.34216513354425
```

Convert to KeplerInputs instance, contains two conversions:

In [5]:
from poliastro.twobody.angles import M_to_E, E_to_nu
import astropy.units as u
from math import pi

In [6]:
# mean anomaly to true anomaly via poliastro functions
ecc = u.Quantity(tle.ecc, unit=u.one)
mean_anom = u.Quantity(tle.M, unit=u.deg)
ecc, mean_anom

(<Quantity 0.0001982>, <Quantity 266.6603 deg>)

In [7]:
mean_anom_rad = mean_anom.to(u.rad)
# mean_anom_rad_negpos = u.Quantity(pi,unit=u.rad) - mean_anom.to(u.rad)
# -pi (when pi passed) + (mean_anom_rad - pi) (overlap over pi)
mean_anom_rad_negpos = mean_anom.to(u.rad) - u.Quantity(2*pi,unit=u.rad) 

mean_anom_rad, mean_anom_rad_negpos, 2*pi + mean_anom_rad_negpos.value

(<Quantity 4.65410022 rad>, <Quantity -1.62908509 rad>, 4.654100219355835)

In [8]:
import tletools

In [9]:
"""
**Warning**

    The mean anomaly must be between -π and π radians.
    The eccentricity must be less than 1.
"""
    
# tle-tools-based true anomaly
true_anom_t = u.Quantity(tletools.utils.M_to_nu(mean_anom_rad_negpos, tle.ecc), unit=u.rad).to(u.deg)
true_anom_t, true_anom_t.to(u.rad)

(<Quantity -93.36237315 deg>, <Quantity -1.62948081 rad>)

In [10]:
# poliastro based True anomaly -> can use mean anomaly in degrees
true_anom_p = E_to_nu( M_to_E( mean_anom_rad_negpos, ecc ), ecc )
_true_anom_p = E_to_nu( M_to_E( mean_anom_rad, ecc ), ecc )
true_anom_p, _true_anom_p, true_anom_p.to(u.deg)

(<Quantity -1.62948081 rad>,
 <Quantity -1.62948081 rad>,
 <Quantity -93.36237315 deg>)

In [11]:
from orbital.utilities import true_anomaly_from_mean

In [12]:
# OrbitalPy based True anomaly
# uses radian in 0,2pi range (Mnorm = fmod(M, 2 * pi))
true_anom_o = true_anomaly_from_mean(ecc.value, mean_anom_rad.value)
true_anom_o, u.Quantity( -1 * (2*pi - true_anom_o), unit=u.rad).to(u.deg)

(4.653704498287206, <Quantity -93.36237315 deg>)

In [13]:
import rebound

In [14]:
# rebound-based True anomaly
# can't really tell in which range M should be, as its C code
true_anom_r = rebound.M_to_f(tle.ecc, mean_anom_rad_negpos.value)
true_anom_r, u.Quantity(-360, unit=u.deg) + u.Quantity(true_anom_r, unit=u.rad).to(u.deg)

(4.653704498287205, <Quantity -93.36237315 deg>)

In [15]:
from skyfield import keplerlib

In [16]:
# can take mean anomalies in both ranges
ecc_anom = keplerlib.eccentric_anomaly(tle.ecc, mean_anom_rad.value)
_ecc_anom = keplerlib.eccentric_anomaly(tle.ecc, mean_anom_rad_negpos.value)
true_anom_s = keplerlib.true_anomaly_closed(tle.ecc, ecc_anom)
_true_anom_s = keplerlib.true_anomaly_closed(tle.ecc, _ecc_anom)

true_anom_s, u.Quantity(true_anom_s, unit=u.rad).to(u.deg), _true_anom_s

(-1.629480808892381, <Quantity -93.36237315 deg>, -1.629480808892381)

Semi-major axis

In [17]:
# tletools 
tle.a

7155.808268901222

In [18]:
from astropy.constants import GM_earth

In [19]:
# researched formula
secs_per_revolution = 86400 / tle.n
semimajoraxis = u.Quantity((secs_per_revolution**2 * GM_earth.value / (4 * pi**2) )**(1/3) / 1000, unit=u.km)
semimajoraxis

<Quantity 7155.80801877 km>

TEME coordinates from poliastro orbit

In [20]:
# creates kepler orbit
orb = tle.to_orbit()
orb

7154 x 7157 km x 86.4 deg (GCRS) orbit around Earth (♁) at epoch 2023-10-22T22:01:33.473856000 (UTC)

Show that equivalent to reference calculations from literature / validated above:

In [21]:
from keplerinputs import KeplerInputs
from poliastro.twobody import Orbit
from poliastro.bodies import Earth

In [22]:
kepler_inputs = KeplerInputs(tle.name, tle.ecc, tle.inc, tle.raan, tle.argp, mean_anom=tle.M, mean_motion=tle.n, epoch=tle.epoch)
orb = Orbit.from_classical(
            Earth, 
            a=kepler_inputs.semimajoraxis, 
            ecc=kepler_inputs.ecc, 
            inc=kepler_inputs.inc, 
            raan=kepler_inputs.raan, 
            argp=kepler_inputs.argp, 
            nu=kepler_inputs.true_anom, 
            epoch=kepler_inputs.epoch)
orb

7154 x 7157 km x 86.4 deg (GCRS) orbit around Earth (♁) at epoch 2023-10-22T22:01:33.473856000 (UTC)

=> tletools orbit creation is equivalent.

Now: compare ephem coordinates to TEME coordinates from C++ SGP4

In [23]:
from datetime import datetime
from astropy.time import Time

In [24]:
# following avg. epoch of all sats, which is encoded in name of TLEs file, 'iridium-NEXT_2023-10-22-21-39-19'
start_datetime = datetime.strptime("2023-10-22-21-39-19", '%Y-%m-%d-%H-%M-%S')
sim_start_time = Time(start_datetime, format='datetime', scale='utc')
sim_start_time

<Time object: scale='utc' format='datetime' value=2023-10-22 21:39:19>

In [25]:
from poliastro.twobody.sampling import EpochsArray
from poliastro.util import time_range

In [26]:
# could use different propagator for EpochsArray, which currently is 'FarnocchiaPropagator'
update_interval = (1 << u.s)
periods = 25
# first coordinate change at first sim second
ephem = orb.to_ephem(strategy=EpochsArray(epochs=time_range(sim_start_time + update_interval, spacing=update_interval, periods=periods)))
ephem

Ephemerides at 25 epochs from 2023-10-22 21:39:20 (UTC) to 2023-10-22 21:39:44 (UTC)

In [27]:
# ICRS coordinates, but orbit above specifies GCRS?
# => GCRS is specific version of ICRS
"""
Regardless of how the Orbit is created, 
the implicit reference system is an inertial one. 
For the specific case of the Solar System, this can be assumed 
to be the International Celestial Reference System or ICRS.
"""

cartesian_trace = ephem.sample(ephem.epochs)
# example coordinates
cartesian_trace[:3]

<CartesianRepresentation (x, y, z) in km
    [(-566.52604985, 1249.35180412, -7024.52179864),
     (-571.61236662, 1254.64088416, -7023.16674788),
     (-576.69806194, 1259.92860018, -7021.80406161)]
 (has differentials w.r.t.: 's')>

In [28]:
from astropy.coordinates import GCRS, ITRS, WGS84GeodeticRepresentation, SkyCoord

In [29]:
# validated: is GCRS
gcrs_trace = GCRS(cartesian_trace, obstime=ephem.epochs)
gcrs_trace.cartesian[:3]

<CartesianRepresentation (x, y, z) in km
    [(-566.52604985, 1249.35180412, -7024.52179864),
     (-571.61236662, 1254.64088416, -7023.16674788),
     (-576.69806194, 1259.92860018, -7021.80406161)]
 (has differentials w.r.t.: 's')>

In [31]:
# convert GCRS to ITRF/S
itrf_trace = gcrs_trace.transform_to(ITRS(obstime=ephem.epochs))
itrf_trace.cartesian[:3]

<CartesianRepresentation (x, y, z) in km
    [(-644.86301954, 1203.58311841, -7025.75814829),
     (-650.25598513, 1208.51251027, -7024.41458634),
     (-655.64752835, 1213.44138109, -7023.06338752)]
 (has differentials w.r.t.: 's')>

In [32]:
# transform to SkyCoord
skycoord_itrf = SkyCoord(itrf_trace, obstime=ephem.epochs, frame="itrs")
skycoord_itrf.cartesian[15:]

<CartesianRepresentation (x, y, z) in km
    [(-725.60553853, 1277.46676615, -7004.80352685),
     (-730.97656521, 1282.3877606 , -7003.34556976),
     (-736.34608341, 1287.30815087, -7001.8799987 ),
     (-741.71408742, 1292.22793144, -7000.40681525),
     (-747.08057153, 1297.14709674, -6998.92602102),
     (-752.44553001, 1302.06564125, -6997.43761762),
     (-757.80895715, 1306.98355941, -6995.94160666),
     (-763.17084724, 1311.90084569, -6994.43798978),
     (-768.53119458, 1316.81749452, -6992.92676859),
     (-773.88999345, 1321.73350039, -6991.40794475)]
 (has differentials w.r.t.: 's')>

For comparison, output by C++ SGP4
```
> print(x_sgp4_itrf$`itrfCoordX_vector`)
 [1] -734.9439 -740.2763 -745.6072 -750.9365 -756.2643 -761.5906 -766.9153 -772.2386 -777.5602 -782.8803 -788.1988
> print(y_sgp4_itrf$`itrfCoordY_vector`)
 [1] 1280.705 1285.646 1290.587 1295.527 1300.466 1305.405 1310.343 1315.280 1320.217 1325.153 1330.089
> print(z_sgp4_itrf$`itrfCoordZ_vector`)
 [1] -7010.836 -7009.370 -7007.896 -7006.415 -7004.926 -7003.430 -7001.926 -7000.414 -6998.895 -6997.368 -6995.834
```

In [196]:
# transform to WGS84
skycoord_wgs84 = skycoord_itrf.copy()
skycoord_wgs84.set_representation_cls(WGS84GeodeticRepresentation)
skycoord_wgs84[15:]

<SkyCoord (ITRS: obstime=[datetime.datetime(2023, 10, 22, 21, 39, 35)
 datetime.datetime(2023, 10, 22, 21, 39, 36)
 datetime.datetime(2023, 10, 22, 21, 39, 37)
 datetime.datetime(2023, 10, 22, 21, 39, 38)
 datetime.datetime(2023, 10, 22, 21, 39, 39)
 datetime.datetime(2023, 10, 22, 21, 39, 40)
 datetime.datetime(2023, 10, 22, 21, 39, 41)
 datetime.datetime(2023, 10, 22, 21, 39, 42)
 datetime.datetime(2023, 10, 22, 21, 39, 43)
 datetime.datetime(2023, 10, 22, 21, 39, 44)], location=(0., 0., 0.) km): (lon, lat, height) in (deg, deg, m)
    [(119.59672477, -78.22323616, 799563.2714196 ),
     (119.68366428, -78.16682804, 799554.57686177),
     (119.7697517 , -78.11039084, 799545.8381859 ),
     (119.85499866, -78.05392496, 799537.05542884),
     (119.93941662, -77.99743081, 799528.22862769),
     (120.02301682, -77.94090878, 799519.35781967),
     (120.10581031, -77.88435925, 799510.44304222),
     (120.18780795, -77.82778259, 799501.48433297),
     (120.26902042, -77.77117919, 799492.481

For comparison, outputs by C++ SGP4:

```
> print(x_sgp4$`wgs84CoordLat_vector`)
 [1] -78.17508 -78.11866 -78.06220 -78.00573 -77.94922 -77.89269 -77.83613 -77.77955 -77.72294 -77.66631 -77.60965
> print(y_sgp4$`wgs84CoordLon_vector`)
 [1] 119.8497 119.9334 120.0162 120.0982 120.1794 120.2599 120.3395 120.4184 120.4966 120.5739 120.6506
> print(z_sgp4$`wgs84CoordAlt_vector`)
 [1] 806989.9 806980.9 806971.8 806962.7 806953.5 806944.3 806935.0 806925.7 806916.4 806907.0 806897.6

```